In [1]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################
# 모델 구조 정의 (학습시 사용한 것과 동일)
############################

class GatedConv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, activation=torch.nn.ReLU()):
        super(GatedConv2d, self).__init__()
        self.feature_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_conv(x)
        m = self.mask_conv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class GatedDeconv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1, activation=torch.nn.ReLU()):
        super(GatedDeconv2d, self).__init__()
        self.feature_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_deconv(x)
        m = self.mask_deconv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class ContextualAttention(torch.nn.Module):
    def __init__(self, kernel_size=3, stride=1, dilation=1):
        super(ContextualAttention, self).__init__()
        self.conv = torch.nn.Conv2d(512, 512, kernel_size, stride, dilation, bias=False)
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        B,C,H,W = x.size()
        query = x.view(B,C,-1)
        key = x.view(B,C,-1)
        value = x.view(B,C,-1)
        attn = torch.bmm(query.permute(0,2,1), key)
        attn = self.softmax(attn)
        out = torch.bmm(attn, value.permute(0,2,1))
        out = out.permute(0,2,1).view(B,C,H,W)
        out = self.conv(out)
        return out

class Stage1Generator(torch.nn.Module):
    def __init__(self):
        super(Stage1Generator, self).__init__()
        self.encoder = torch.nn.Sequential(
            GatedConv2d(4, 64, 4, 2, 1),
            GatedConv2d(64, 128, 4, 2, 1),
            GatedConv2d(128, 256, 4, 2, 1),
            GatedConv2d(256, 512, 4, 2, 1)
        )
        self.decoder = torch.nn.Sequential(
            GatedDeconv2d(512, 256, 4, 2, 1),
            GatedDeconv2d(256, 128, 4, 2, 1),
            GatedDeconv2d(128, 64, 4, 2, 1),
            GatedDeconv2d(64, 64, 4, 2, 1, activation=torch.nn.ReLU()),
            torch.nn.Conv2d(64, 3, 3, 1, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x, mask):
        inp = torch.cat((x, mask), dim=1)
        feat = self.encoder(inp)
        out = self.decoder(feat)
        return out

class Stage2Generator(torch.nn.Module):
    def __init__(self):
        super(Stage2Generator, self).__init__()
        self.encoder = torch.nn.Sequential(
            GatedConv2d(7, 64, 4, 2, 1),
            GatedConv2d(64, 128, 4, 2, 1),
            GatedConv2d(128, 256, 4, 2, 1),
            GatedConv2d(256, 512, 4, 2, 1)
        )
        self.contextual_attention = ContextualAttention()
        self.decoder = torch.nn.Sequential(
            GatedDeconv2d(512, 256, 4, 2, 1),
            GatedDeconv2d(256, 128, 4, 2, 1),
            GatedDeconv2d(128, 64, 4, 2, 1),
            GatedDeconv2d(64, 64, 4, 2, 1, activation=torch.nn.ReLU()),
            torch.nn.Conv2d(64, 3, 3, 1, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, coarse_out, inp, mask):
        fin_inp = torch.cat((coarse_out, inp, mask), dim=1)
        feat = self.encoder(fin_inp)
        feat = self.contextual_attention(feat)
        out = self.decoder(feat)
        return out

# 최적 모델 가중치 경로
best_coarse_path = "best_coarse_generator_epoch5.pth"
best_fine_path = "best_fine_generator_epoch5.pth"


# 테스트 이미지(손상된 컬러 이미지) 폴더
test_input_dir = "../02_color/data/output_grayTocol_2024123101"
test_mask_dir = "../data/output_01_mask"
output_dir = "data/output_colToper_2024123101"

os.makedirs(output_dir, exist_ok=True)

coarse_generator = Stage1Generator().to(device)
fine_generator = Stage2Generator().to(device)

coarse_generator.load_state_dict(torch.load(best_coarse_path, map_location=device, weights_only=True))
fine_generator.load_state_dict(torch.load(best_fine_path, map_location=device, weights_only=True))

coarse_generator.eval()
fine_generator.eval()

transform = T.Compose([
    T.Resize((512,512)),
    T.ToTensor()
])

test_files = [f for f in os.listdir(test_input_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]

with torch.no_grad():
    for filename in test_files:
        input_path = os.path.join(test_input_dir, filename)
        mask_path = os.path.join(test_mask_dir, filename)

        if not os.path.exists(mask_path):
            print(f"{mask_path}가 존재하지 않습니다. 스킵합니다.")
            continue

        inp_img = Image.open(input_path).convert("RGB")
        mask_img = Image.open(mask_path).convert("L")

        inp_tensor = transform(inp_img).unsqueeze(0).to(device)   # (1,3,H,W)
        mask_tensor = transform(mask_img).unsqueeze(0).to(device) # (1,1,H,W)

        # 손상 영역 0 처리
        mask_broadcast = mask_tensor.expand_as(inp_tensor)
        damaged_inp = inp_tensor * (1 - mask_broadcast)

        # 복원
        coarse_out = coarse_generator(damaged_inp, mask_tensor)
        fine_out = fine_generator(coarse_out, damaged_inp, mask_tensor)

        # 여기서 복원된 부분(fine_out)을 마스크가 1인 영역에만 적용
        # final_result = original_damaged_image * (1 - mask) + fine_out * mask
        final_result = inp_tensor * (1 - mask_broadcast) + fine_out * mask_broadcast

        # 결과 텐서를 이미지로 변환
        final_result_pil = T.ToPILImage()(final_result.squeeze(0).cpu())

        save_path = os.path.join(output_dir, filename)
        final_result_pil.save(save_path)
        print(f"{filename} 복원 완료(마스크 영역만 덮어쓰기) -> {save_path}")


TEST_000.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_000.png
TEST_001.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_001.png
TEST_002.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_002.png
TEST_003.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_003.png
TEST_004.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_004.png
TEST_005.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_005.png
TEST_006.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_006.png
TEST_007.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_007.png
TEST_008.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_008.png
TEST_009.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_009.png
TEST_010.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_010.png
TEST_011.png 복원 완료(마스크 영역만 덮어쓰기) -> data/output_colToper_2024123101\TEST_011.png
TEST_012.png 복원 완료(마스크 영역만 덮

# 마스크 확장버전

In [5]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import cv2  # 마스크 확장용
###############################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

############################
# 모델 구조 정의 (학습시 사용한 것과 동일)
############################
class GatedConv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, activation=torch.nn.ReLU()):
        super(GatedConv2d, self).__init__()
        self.feature_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_conv(x)
        m = self.mask_conv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class GatedDeconv2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1, activation=torch.nn.ReLU()):
        super(GatedDeconv2d, self).__init__()
        self.feature_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.mask_deconv = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = torch.nn.Sigmoid()
        self.activation = activation

    def forward(self, x):
        f = self.feature_deconv(x)
        m = self.mask_deconv(x)
        gated = self.sigmoid(m)
        if self.activation is not None:
            f = self.activation(f)
        return f * gated

class ContextualAttention(torch.nn.Module):
    def __init__(self, kernel_size=3, stride=1, dilation=1):
        super(ContextualAttention, self).__init__()
        self.conv = torch.nn.Conv2d(512, 512, kernel_size, stride, dilation, bias=False)
        self.softmax = torch.nn.Softmax(dim=-1)
    def forward(self, x):
        B,C,H,W = x.size()
        query = x.view(B,C,-1)
        key = x.view(B,C,-1)
        value = x.view(B,C,-1)
        attn = torch.bmm(query.permute(0,2,1), key)
        attn = self.softmax(attn)
        out = torch.bmm(attn, value.permute(0,2,1))
        out = out.permute(0,2,1).view(B,C,H,W)
        out = self.conv(out)
        return out

class Stage1Generator(nn.Module):
    def __init__(self):
        super().__init__()

        # ----- Encoder (Down) -----
        self.enc1 = GatedConv2d(4, 64, 4, 2, 1)
        self.enc2 = GatedConv2d(64, 128, 4, 2, 1)
        self.enc3 = GatedConv2d(128, 256, 4, 2, 1)
        self.enc4 = GatedConv2d(256, 512, 4, 2, 1)

        # ----- Decoder (Up) -----
        # 1) e4 -> d1, (H/16 -> H/8)
        #    cat with e3 → (256 + 256) -> dec1_2
        self.dec1_up = GatedDeconv2d(512, 256, 4, 2, 1)
        self.dec1_conv = GatedConv2d(256 + 256, 256, 3, 1, 1)

        # 2) d1 -> d2, (H/8 -> H/4)
        #    cat with e2 → (256 + 128) -> dec2_2
        self.dec2_up = GatedDeconv2d(256, 128, 4, 2, 1)
        self.dec2_conv = GatedConv2d(128 + 128, 128, 3, 1, 1)

        # 3) d2 -> d3, (H/4 -> H/2)
        #    cat with e1 → (128 + 64) -> dec3_2
        self.dec3_up = GatedDeconv2d(128, 64, 4, 2, 1)
        self.dec3_conv = GatedConv2d(64 + 64, 64, 3, 1, 1)

        # 4) d3 -> d4, (H/2 -> H)
        #    여기서는 skip이 inp(4ch)일 수도, e0(3ch+mask)일 수도 있지만 보통 e1까지만.
        self.dec4_up = GatedDeconv2d(64, 64, 4, 2, 1)
        self.dec4_conv = nn.Conv2d(64, 3, 3, 1, 1)
        self.final_act = nn.Sigmoid()

    def forward(self, x, mask):
        # ----- Encoder -----
        inp = torch.cat((x, mask), dim=1)  # (N, 4, H, W)
        e1 = self.enc1(inp)    # (N,64,  H/2,  W/2)
        e2 = self.enc2(e1)     # (N,128, H/4,  W/4)
        e3 = self.enc3(e2)     # (N,256, H/8,  W/8)
        e4 = self.enc4(e3)     # (N,512, H/16, W/16)

        # ----- Decoder -----
        # 1) 업샘플 e4 -> d1_up (H/16 -> H/8), cat e3
        d1_up = self.dec1_up(e4)              # (N,256, H/8, W/8)
        d1_in = torch.cat([d1_up, e3], dim=1) # (N,256+256=512, H/8, W/8)
        d1 = self.dec1_conv(d1_in)           # (N,256, H/8, W/8)

        # 2) 업샘플 d1 -> d2_up (H/8 -> H/4), cat e2
        d2_up = self.dec2_up(d1)              # (N,128, H/4, W/4)
        d2_in = torch.cat([d2_up, e2], dim=1) # (N,128+128=256, H/4, W/4)
        d2 = self.dec2_conv(d2_in)           # (N,128, H/4, W/4)

        # 3) 업샘플 d2 -> d3_up (H/4 -> H/2), cat e1
        d3_up = self.dec3_up(d2)              # (N,64, H/2, W/2)
        d3_in = torch.cat([d3_up, e1], dim=1) # (N,64+64=128, H/2, W/2)
        d3 = self.dec3_conv(d3_in)           # (N,64, H/2, W/2)

        # 4) 업샘플 d3 -> d4_up (H/2 -> H)
        d4_up = self.dec4_up(d3)             # (N,64, H, W)
        d4 = self.dec4_conv(d4_up)           # (N,3,  H, W)

        out = self.final_act(d4)
        return out



class Stage2Generator(nn.Module):
    def __init__(self):
        super(Stage2Generator, self).__init__()

        # ------------------------
        # Encoder
        # ------------------------
        self.enc1 = GatedConv2d(7,   64, 4, 2, 1)  # (N, 64,  H/2,  W/2)
        self.enc2 = GatedConv2d(64,  128, 4, 2, 1) # (N, 128, H/4,  W/4)
        self.enc3 = GatedConv2d(128, 256, 4, 2, 1) # (N, 256, H/8,  W/8)
        self.enc4 = GatedConv2d(256, 512, 4, 2, 1) # (N, 512, H/16, W/16)

        # Contextual Attention (옵션)
        self.contextual_attention = ContextualAttention()

        # ------------------------
        # Decoder (U-Net style)
        #   - dec1_up: e4(H/16→H/8)
        #   - dec1_conv: concat with e3 → (256+256)
        #   - dec2_up: d1(H/8→H/4)
        #   - dec2_conv: concat with e2 → (128+128)
        #   - ...
        # ------------------------

        # 1) H/16 -> H/8
        self.dec1_up   = GatedDeconv2d(512, 256, 4, 2, 1)
        self.dec1_conv = GatedConv2d(256 + 256, 256, 3, 1, 1)

        # 2) H/8 -> H/4
        self.dec2_up   = GatedDeconv2d(256, 128, 4, 2, 1)
        self.dec2_conv = GatedConv2d(128 + 128, 128, 3, 1, 1)

        # 3) H/4 -> H/2
        self.dec3_up   = GatedDeconv2d(128, 64, 4, 2, 1)
        self.dec3_conv = GatedConv2d(64 + 64, 64, 3, 1, 1)

        # 4) H/2 -> H
        self.dec4_up   = GatedDeconv2d(64, 64, 4, 2, 1)
        self.dec4_conv = nn.Conv2d(64, 3, 3, 1, 1)

        self.final_act = nn.Sigmoid()

    def forward(self, coarse_out, inp, mask):
        # (N, 3, H, W), (N,3,H,W), (N,1,H,W)
        fin_inp = torch.cat((coarse_out, inp, mask), dim=1) # (N,7,H,W)

        # ----- Encoder -----
        e1 = self.enc1(fin_inp)  # (N,64,  H/2,  W/2)
        e2 = self.enc2(e1)       # (N,128, H/4,  W/4)
        e3 = self.enc3(e2)       # (N,256, H/8,  W/8)
        e4 = self.enc4(e3)       # (N,512, H/16, W/16)

        # Contextual Attention (옵션)
        e4_attn = self.contextual_attention(e4)  # (N,512, H/16, W/16)

        # ----- Decoder -----
        # 1) 업샘플 e4_attn -> d1_up(H/8), concat e3
        d1_up = self.dec1_up(e4_attn)            # (N,256, H/8, W/8)
        d1_in = torch.cat([d1_up, e3], dim=1)    # (N,256+256=512, H/8, W/8)
        d1 = self.dec1_conv(d1_in)              # (N,256, H/8, W/8)

        # 2) 업샘플 d1 -> d2_up(H/4), concat e2
        d2_up = self.dec2_up(d1)                 # (N,128, H/4, W/4)
        d2_in = torch.cat([d2_up, e2], dim=1)    # (N,128+128=256, H/4, W/4)
        d2 = self.dec2_conv(d2_in)              # (N,128, H/4, W/4)

        # 3) 업샘플 d2 -> d3_up(H/2), concat e1
        d3_up = self.dec3_up(d2)                 # (N,64,  H/2, W/2)
        d3_in = torch.cat([d3_up, e1], dim=1)    # (N,64+64=128, H/2, W/2)
        d3 = self.dec3_conv(d3_in)              # (N,64,  H/2, W/2)

        # 4) 업샘플 d3 -> d4_up(H), 최종
        d4_up = self.dec4_up(d3)                 # (N,64,  H, W)
        d4    = self.dec4_conv(d4_up)            # (N,3,   H, W)

        out = self.final_act(d4)
        return out

# 최적 모델 가중치 경로
best_coarse_path = "model/05/best_coarse_generator_epoch5.pth"
best_fine_path   = "model/05/best_fine_generator_epoch5.pth"

# 테스트 이미지(손상된 컬러 이미지) 폴더
test_input_dir = "../02_color/data/output_grayTocol_2025010201_nowbest"
test_mask_dir  = "../data/output_01_mask"
output_dir     = "data/output_colToper_2025010204"
os.makedirs(output_dir, exist_ok=True)

coarse_generator = Stage1Generator().to(device)
fine_generator   = Stage2Generator().to(device)

coarse_generator.load_state_dict(torch.load(best_coarse_path, map_location=device, weights_only=True))
fine_generator.load_state_dict(torch.load(best_fine_path,   map_location=device, weights_only=True))

coarse_generator.eval()
fine_generator.eval()

transform = T.Compose([
    T.Resize((512,512)),
    T.ToTensor()
])

test_files = [f for f in os.listdir(test_input_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]

with torch.no_grad():
    for filename in test_files:
        input_path = os.path.join(test_input_dir, filename)
        mask_path  = os.path.join(test_mask_dir, filename)

        if not os.path.exists(mask_path):
            print(f"{mask_path}가 존재하지 않습니다. 스킵합니다.")
            continue

        # (1) 입력 이미지 로드
        inp_img = Image.open(input_path).convert("RGB")
        # (2) 마스크 이미지 로드
        mask_img = Image.open(mask_path).convert("L")

        # ========== (A) 마스크 확장(팽창) 과정 추가 ==========
        mask_np = np.array(mask_img, dtype=np.uint8)
        kernel = np.ones((3,3), np.uint8)   # 3x3 커널
        dilated_mask_np = cv2.dilate(mask_np, kernel, iterations=1)  # 1회 팽창
        # 다시 PIL로 변환
        mask_img = Image.fromarray(dilated_mask_np)

        # 변환
        inp_tensor = transform(inp_img).unsqueeze(0).to(device)   # (1,3,H,W)
        mask_tensor = transform(mask_img).unsqueeze(0).to(device) # (1,1,H,W)

        # 손상 영역 0 처리
        mask_broadcast = mask_tensor.expand_as(inp_tensor)
        damaged_inp = inp_tensor * (1.0 - mask_broadcast)

        # (3) Coarse 복원
        coarse_out = coarse_generator(damaged_inp, mask_tensor)
        # (4) Fine 복원
        fine_out = fine_generator(coarse_out, damaged_inp, mask_tensor)

        # (5) 최종 결과: 마스크 부분은 fine_out, 나머지는 원본
        final_result = inp_tensor * (1.0 - mask_broadcast) + fine_out * mask_broadcast

        # (6) 결과 저장
        final_result_pil = T.ToPILImage()(final_result.squeeze(0).cpu())
        save_path = os.path.join(output_dir, filename)
        final_result_pil.save(save_path)
        print(f"{filename} 복원 완료(마스크 확장 적용) -> {save_path}")


TEST_000.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_000.png
TEST_001.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_001.png
TEST_002.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_002.png
TEST_003.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_003.png
TEST_004.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_004.png
TEST_005.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_005.png
TEST_006.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_006.png
TEST_007.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_007.png
TEST_008.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_008.png
TEST_009.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_009.png
TEST_010.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_010.png
TEST_011.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204\TEST_011.png
TEST_012.png 복원 완료(마스크 확장 적용) -> data/output_colToper_2025010204